# MNIST Classification with Classic Features

This notebook demonstrates how to classify the MNIST handwritten digits using classical computer vision features. We extract:

* **Object analysis features** (e.g., area, aspect ratio, eccentricity, solidity, Hu moments) from binarized digits.
* **Texture features** via Local Binary Patterns (LBP).

The combined descriptors feed into a one-vs-all Support Vector Machine (SVM) classifier.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from skimage.feature import local_binary_pattern
from skimage.measure import label, regionprops, moments_hu

np.random.seed(42)


## Load the MNIST dataset

We load the images and labels from `fetch_openml` and optionally down-sample for quicker experimentation. Images are kept as 28×28 grayscale arrays for feature extraction.


In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
images = mnist.data.reshape(-1, 28, 28).astype(np.float32) / 255.0
labels = mnist.target.astype(int)

n_samples = 20000  # adjust down if running on limited hardware
indices = np.random.choice(len(images), size=n_samples, replace=False)
images = images[indices]
labels = labels[indices]

print(f"Dataset subset: {images.shape[0]} samples")


## Feature extraction helpers

* **Object analysis**: binarize the digit, find connected components, and compute shape attributes.
* **Texture**: compute uniform LBP and summarize it with a normalized histogram.
* **Combined descriptor**: concatenate object and texture vectors for each image.


In [ ]:
def extract_object_features(image: np.ndarray) -> np.ndarray:
    binary = image > 0.2
    labeled = label(binary)
    props = regionprops(labeled)
    if not props:
        return np.zeros(12, dtype=np.float32)
    region = max(props, key=lambda p: p.area)
    area = region.area / (image.shape[0] * image.shape[1])
    eccentricity = region.eccentricity
    extent = region.extent
    solidity = region.solidity
    minr, minc, maxr, maxc = region.bbox
    height, width = maxr - minr, maxc - minc
    aspect_ratio = width / height if height > 0 else 0.0
    hu = moments_hu(binary.astype(np.float64))
    hu = np.sign(hu) * np.log1p(np.abs(hu))
    return np.hstack([
        area,
        aspect_ratio,
        eccentricity,
        extent,
        solidity,
        hu,
    ]).astype(np.float32)


def extract_texture_features(image: np.ndarray, radius: int = 2, n_points: int | None = None) -> np.ndarray:
    if n_points is None:
        n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), range=(0, n_points + 2), density=True)
    return hist.astype(np.float32)


def extract_features(image: np.ndarray) -> np.ndarray:
    obj_feats = extract_object_features(image)
    tex_feats = extract_texture_features(image)
    return np.hstack([obj_feats, tex_feats])


## Build the feature matrix

Compute the descriptors for every image. We keep a few examples to visualize the binary masks and LBP responses for intuition.


In [ ]:
feature_list = [extract_features(img) for img in images]
X = np.vstack(feature_list)
y = labels

print(f"Feature matrix shape: {X.shape}")


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for ax, idx in zip(axes.flat, range(5)):
    img = images[idx]
    binary = img > 0.2
    lbp = local_binary_pattern(img, 16, 2, method="uniform")
    ax.imshow(np.hstack([img, binary, lbp / lbp.max()]), cmap='gray')
    ax.set_title(f"Label: {labels[idx]}")
    ax.axis('off')
plt.suptitle("Example digits with binary mask and LBP")
plt.tight_layout()
plt.show()


## Train a one-vs-all SVM classifier

We standardize the feature matrix and fit a linear SVM. `LinearSVC` uses the one-vs-all strategy by default, which suits the multi-class MNIST problem.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

svm_clf = make_pipeline(
    StandardScaler(),
    LinearSVC(dual=False, random_state=42)
)
svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))


## Visualize the confusion matrix

The confusion matrix highlights which digits remain challenging for the classical feature set.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='Blues', normalize='true', ax=ax)
ax.set_title("Linear SVM (one-vs-all) on Object + Texture Features")
plt.tight_layout()
plt.show()
